# Wykład 11: Konwolucje (Convolutions)

Cel wykładu:
- Zastosowanie konwolucji do obrazów

### Wczytywanie i wyświetlanie obrazów

Kod poniżej wczytuje obrazek umieszcza go w tablicy `numpy` i wyświetla w notatniku.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from skimage.io import imread
from skimage.transform import resize

In [ ]:
sample_image = imread("pwr.jpg")
sample_image= sample_image.astype("float32")

size = sample_image.shape
print("sample image shape: ", sample_image.shape)

plt.figure(figsize=(15,15))
plt.imshow(sample_image.astype('uint8'));

### Prosty filtr konwolucyjny (convolution filter)

Wykorzystamy Keras'a (tensorflow) do wykonywania konwolucji na obrazku. **Nie będziemy na razie przeprowadzać uczenia w tych modelach celem jest wizualne zrozumienie działania konwolucji**

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D

In [ ]:
Conv2D?

In [ ]:
conv = Conv2D(filters=3, kernel_size=(5, 5), padding="same",
              input_shape=(None, None, 3))

**Uwaga**: w Kerasie, `None` jest wykorzystywany do wskazania, że wymiar tensora ma dynamiczny rozmiar.
W tym przypadku `batch_size`, `width` i `height` są wszystkie dynamiczne: czyli po prostu zależą od wejścia (input). Tylko liczba kanałów wejściowych (input channels) jest ustalona na 3 oraz wielkość `kernel` jest (5,5).

In [ ]:
sample_image.shape

In [ ]:
img_in = np.expand_dims(sample_image, 0)
img_in.shape

## Wynik zastosowania konwolucji do obrazka 

In [ ]:
img_out = conv(img_in)
print(type(img_out), img_out.shape)

Wyjściem `Conv2D` jest klasa tensorflow Eager Tensor, którą możemy przekonwerterować do standardowej
tablicy `numpy`

In [ ]:
np_img_out = img_out[0].numpy()
print(type(np_img_out))

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(15, 25))
ax0.set_title('Oryginalny obrazek')
ax1.set_title('Konwoulcja')
ax0.imshow(sample_image.astype('uint8'))
ax1.imshow(np_img_out.astype('uint8'));
fig.tight_layout(pad=0.05)
plt.subplots_adjust(bottom=0.175, wspace=0.05)


## Analiza

Wyjście ma 3 kanały, możemy to interpretować jako obrazek RGB. Zauważmy, że za każdym razem wykonując
Conv2D (kownolucje) dostaniemy inny wynik. Domyślnie filtr (podobnie jak wagi w sieciach feedforword)
inicjalizowany jest wartościami losowymi.

Zobaczmy, więc parametry

In [ ]:
conv.count_params()

In [ ]:
conv.weights

Każdy z 3 kanałów wyjść jest generowany przez inne jądro konwolucji (convolution kernel).
Każde jądro konwolucyjne (convolution kernel) ma wymiar 5x5 i operuje na 3 wejściach kanałów.

In [ ]:
len(conv.get_weights())

In [ ]:
weights = conv.get_weights()[0]
weights.shape

In [ ]:
biases = conv.get_weights()
#biases.shape
biases

Jedno odchylenie (bias) na kanał wyjściowy.

Zamiast tego możemy zbudować jądro, definiując funkcję, która zostanie przekazana do warstwy `Conv2D`.
Stworzymy tablicę z wartościami np. 1/45 dla filtrów, z każdym kanałem oddzielnie.

In [ ]:
def my_init(shape=(5, 5, 3, 3), dtype=None):
    array = np.zeros(shape=shape, dtype="float32")
    array[:, :, 0, 0] = 1 / 45
    array[:, :, 1, 1] = 1 / 45
    array[:, :, 2, 2] = 1 / 45
    return array

Możemy wyświetlić filtry numpy, przesuwając wymiary przestrzenne na końcu (używając `np.transpose`):

In [ ]:
np.transpose(my_init(), (2, 3, 0, 1))

In [ ]:
conv = Conv2D(filters=3, kernel_size=(5, 5), padding="same",
           input_shape=(None, None, 3), kernel_initializer=my_init)

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(15, 25))
ax0.set_title('Oryginalny obrazek')
ax1.set_title('Konwoulcja')

ax0.imshow(img_in[0].astype('uint8'))

img_out = conv(img_in)
np_img_out = img_out[0].numpy()
ax1.imshow(np_img_out.astype('uint8'));
fig.tight_layout(pad=0.05)
plt.subplots_adjust(bottom=0.175, wspace=0.05)


- Zdefiniujemy warstwę Conv2D z 3 filtrami (5x5), które obliczają funkcję tożsamości (zachowajmy obraz wejściowy bez mieszania kolorów).
- Zmienimy `strade` na 2. Zobacz jaki jest rozmiar obrazu wyjściowego (patrz na osie)
- Zmienimy wypełnienie na „VALID”. Co obserwujesz? (patrz na print-y)


In [ ]:
def my_init(shape=(5, 5, 3, 3), dtype=None):
    array = np.zeros(shape=shape, dtype="float32")
    array[2, 2] = np.eye(3)
    return array


conv_strides_same = Conv2D(filters=3, kernel_size=5, strides=2,
           padding="same", kernel_initializer=my_init,
           input_shape=(None, None, 3))

conv_strides_valid = Conv2D(filters=3, kernel_size=5, strides=2,
           padding="valid", kernel_initializer=my_init,
           input_shape=(None, None, 3))

img_in = np.expand_dims(sample_image, 0)
img_out_same = conv_strides_same(img_in)[0].numpy()
img_out_valid = conv_strides_valid(img_in)[0].numpy()

print("Shape of original image:", sample_image.shape)
print("Shape of result with SAME padding:", img_out_same.shape)
print("Shape of result with VALID padding:", img_out_valid.shape)

fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 36))
ax0.imshow(img_in[0].astype(np.uint8))
ax1.imshow(img_out_same.astype(np.uint8))
ax2.imshow(img_out_valid.astype(np.uint8))


## Analiza

Obserwujemy, że `stride` dzieli rozmiar obrazu przez 2
W przypadku trybu padding „VALID”, padding nie jest dodawany, więc
rozmiar wyjściowego obrazu jest w rzeczywistości o 2 mniejszy z powodu
rozmiar jądra

### Wykrywanie krawędzi (edge detection) na obrazkach monochromatycznych

In [ ]:
# convert image to greyscale
grey_sample_image = sample_image.mean(axis=2)

# add the channel dimension even if it's only one channel so
# as to be consistent with Keras expectations.
grey_sample_image = grey_sample_image[:, :, np.newaxis]


# matplotlib does not like the extra dim for the color channel
# when plotting gray-level images. Let's use squeeze:
plt.figure(figsize=(15,15))
plt.imshow(np.squeeze(grey_sample_image.astype(np.uint8)),
           cmap=plt.cm.gray);

Zbudujemy detektor krawędzi za pomocą `Conv2D` na obrazie w skali szarości
- Możesz eksperymentować z kilkoma jądrami, aby znaleźć sposób na wykrycie krawędzi
- https://en.wikipedia.org/wiki/Kernel_(image_processing)

Spróbuj `Conv2D?` 'Lub naciśnij` shift-tab`, aby uzyskać dokumentację. Możesz uzyskać pomoc na https://keras.io/layers/convolutional/

In [ ]:
Conv2D?

In [ ]:
def my_init(shape, dtype=None):
    array = np.array([
        [0.0,  0.2, 0.0],
        [0.0, -0.2, 0.0],
        [0.0,  0.0, 0.0],
    ], dtype="float32")
    # adds two axis to match the required shape (3,3,1,1)
    return np.expand_dims(np.expand_dims(array,-1),-1)


conv_edge = Conv2D(kernel_size=(3,3), filters=1,
           padding="same", kernel_initializer=my_init,
           input_shape=(None, None, 1))

img_in = np.expand_dims(grey_sample_image, 0)
img_out = conv_edge(img_in).numpy()

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15, 15))
ax0.imshow(np.squeeze(img_in[0]).astype(np.uint8),
           cmap=plt.cm.gray);
ax1.imshow(np.squeeze(img_out[0]).astype(np.uint8),
           cmap=plt.cm.gray);



## Analiza

Pokazujemy tutaj tylko wykrywanie krawędzi pionowej.
Wiele innych jąder działa, na przykład "różnice
wyśrodkowanych gaussów" (centred gaussian)

Możesz spróbować również z tym filtrem
```python
np.array ([
         [0,1, 0,2, 0,1],
         [0,0, 0,0, 0,0],
         [-0,1, -0,2, -0,1],
   ], dtype = "float32")
```

# Pooling i strides z konwolucją

- Użyjemy `MaxPool2D`, aby zastosować maksymalną pulę (max pool) 2x2 z krokiem (strides) 2 do obrazu. Jaki jest wpływ na kształt obrazu?
- Użyjemy `AvgPool2D`, aby zastosować średnią pulę (average pooling).
- Czy można obliczyć maksymalną pulę i średnią pulę przy dobrze dobranych jądrach?

**Dodatek**
- Wdrożenie puli średniej 3x3 z regularnym splotem `Conv2D`, z dobrze dobranymi krokami, jądrem i paddingiem

In [ ]:
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [ ]:
# tutaj dałem trochę duże wartości, aby było "widać" efekt pool_size=10, strides=20
# poeksperymentuj z innymi wartościami!

max_pool = MaxPool2D(pool_size=10, strides=20, input_shape=(None, None, 3))
img_in = np.expand_dims(sample_image, 0)
img_out = max_pool(img_in).numpy()

print("input shape:", img_in.shape)
print("output shape:", img_out.shape)

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(15, 25))
ax0.set_title('Oryginalny obrazek')
ax1.set_title('MaxPool')
ax0.imshow(img_in[0].astype('uint8'))
ax1.imshow(img_out[0].astype('uint8'));


**Uwaga:** Możliwe jest zbudowanie `max pool` wykorzystując standardową konwolucje `Conv2D`. Podobnie możliwe jest zbudowanie `avg pool` z odpowiednio dobranymi parametrami

In [ ]:
avg_pool = AvgPool2D(3, strides=3, input_shape=(None, None, 3))

img_in = np.expand_dims(sample_image, 0)
img_out_avg_pool = avg_pool(img_in).numpy()

Ta sama operacja zaimplementowana za pomocą konwolucji

In [ ]:
def my_init(shape=(3, 3, 3, 3), dtype=None):
    array = np.zeros(shape=shape, dtype="float32")
    array[:, :, 0, 0] = 1 / 9.
    array[:, :, 1, 1] = 1 / 9.
    array[:, :, 2, 2] = 1 / 9.
    return array

conv_avg = Conv2D(kernel_size=3, filters=3, strides=3,
           padding="valid", kernel_initializer=my_init,
           input_shape=(None, None, 3))

img_out_conv = conv_avg(np.expand_dims(sample_image, 0)).numpy()

In [ ]:
print("input shape:", img_in.shape)
print("output avg pool shape:", img_out_avg_pool.shape)
print("output conv shape:", img_out_conv.shape)

fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 35))
ax0.imshow(img_in[0].astype('uint8'))
ax1.imshow(img_out_avg_pool[0].astype('uint8'))
ax2.imshow(img_out_conv[0].astype('uint8'));

# zauważ dostajemy że "prawie" to samo!
print("Avg pool is similar to Conv ? -", np.allclose(img_out_avg_pool, img_out_conv))